In [1]:
import pandas as pd
import numpy as np
import random as rnd
import pprint
from itertools import cycle, islice
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
def compute_daily_averages(frame,x,z):
    y = frame.groupby(['HADM_ID','SUBJECT_ID']).agg({x: ['count']})
    y.columns = ['count']
    y = y.reset_index()
    var1 = z.merge(y,on = ['HADM_ID','SUBJECT_ID'],how='left')
    return var1

In [3]:
#reading all the tables required 

admissions = pd.read_csv(r'Z:\mimic-iii-clinical-database-1.4\mimic-iii-clinical-database-1.4\ADMISSIONS.csv\ADMISSIONS.csv')
d_icd_proc = pd.read_csv(r'Z:\mimic-iii-clinical-database-1.4\mimic-iii-clinical-database-1.4\D_ICD_PROCEDURES.csv\D_ICD_PROCEDURES.csv')
proc_icd = pd.read_csv(r'Z:\mimic-iii-clinical-database-1.4\mimic-iii-clinical-database-1.4\PROCEDURES_ICD.csv\PROCEDURES_ICD.csv')
cpt = pd.read_csv(r'Z:\mimic-iii-clinical-database-1.4\mimic-iii-clinical-database-1.4\CPTEVENTS.csv\CPTEVENTS.csv')
transfers = pd.read_csv(r'Z:\mimic-iii-clinical-database-1.4\mimic-iii-clinical-database-1.4\TRANSFERS.csv\TRANSFERS.csv')
lab = pd.read_csv(r'Z:\mimic-iii-clinical-database-1.4\mimic-iii-clinical-database-1.4\LABEVENTS.csv\LABEVENTS.csv')
microLab = pd.read_csv(r'Z:\mimic-iii-clinical-database-1.4\mimic-iii-clinical-database-1.4\MICROBIOLOGYEVENTS.csv\MICROBIOLOGYEVENTS.csv')
noteEvents = pd.read_csv(r'Z:\mimic-iii-clinical-database-1.4\mimic-iii-clinical-database-1.4\NOTEEVENTS.csv\NOTEEVENTS.csv')
outputEvents = pd.read_csv(r'Z:\mimic-iii-clinical-database-1.4\mimic-iii-clinical-database-1.4\OUTPUTEVENTS.csv\OUTPUTEVENTS.csv')
diag_icd = pd.read_csv(r'Z:\mimic-iii-clinical-database-1.4\mimic-iii-clinical-database-1.4\DIAGNOSES_ICD.csv\DIAGNOSES_ICD.csv')
d_icd_diag = pd.read_csv(r'Z:\mimic-iii-clinical-database-1.4\mimic-iii-clinical-database-1.4\D_ICD_DIAGNOSES.csv\D_ICD_DIAGNOSES.csv')
patients = pd.read_csv(r'Z:\mimic-iii-clinical-database-1.4\mimic-iii-clinical-database-1.4\PATIENTS.csv\PATIENTS.csv')



Z:\ProgramData\envs\Tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4,5,7,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
Z:\ProgramData\envs\Tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
#removing the non-essential columns in admissions table
notnecessary = ['EDREGTIME','EDOUTTIME','DEATHTIME','ROW_ID','HAS_CHARTEVENTS_DATA']
for col in notnecessary:
    del admissions[col]

In [5]:
#imputing the important columns in admissions with UNKNOWN
impColumns = ['LANGUAGE','MARITAL_STATUS','RELIGION']
for col in impColumns:
    admissions[col] = admissions[col].fillna("UNKNOWN")

In [6]:
#computing the days difference between admission time and discharge time

from datetime import timedelta
admissions["ADMITTIME"]=pd.to_datetime(admissions["ADMITTIME"])
admissions["DISCHTIME"]=pd.to_datetime(admissions["DISCHTIME"])

admissions["DAYS_DIFF"]=admissions["DISCHTIME"]-admissions["ADMITTIME"]
admissions["DAYS_DIFF"]=admissions["DAYS_DIFF"].apply(timedelta.total_seconds).astype(int)

admissions['DAYS_DIFF'] = admissions['DAYS_DIFF']/(60 * 60 * 24)
admissions['DAYS_DIFF'] = round(admissions['DAYS_DIFF'],2)

In [7]:
admissions

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,DAYS_DIFF
0,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,UNKNOWN,UNOBTAINABLE,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,0,1.14
1,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,UNKNOWN,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,5.50
2,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,BRAIN MASS,0,6.77
3,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,UNKNOWN,PROTESTANT QUAKER,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION,0,2.86
4,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,UNKNOWN,UNOBTAINABLE,MARRIED,WHITE,ACUTE CORONARY SYNDROME,0,3.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,98800,191113,2131-03-30 21:13:00,2131-04-02 15:02:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,NOT SPECIFIED,SINGLE,WHITE,TRAUMA,0,2.74
58972,98802,101071,2151-03-05 20:00:00,2151-03-06 09:10:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,SAH,1,0.55
58973,98805,122631,2200-09-12 07:15:00,2200-09-20 12:08:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,RENAL CANCER/SDA,0,8.20
58974,98813,170407,2128-11-11 02:29:00,2128-12-22 13:11:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,ENGL,CATHOLIC,MARRIED,WHITE,S/P FALL,0,41.45


In [8]:
#locating the days difference which are less than or equal to 2, according to the requirement
idx = np.where((admissions['DAYS_DIFF']<=2) & (admissions['DAYS_DIFF']>0))

In [9]:
#picking out the specific rows mentioned in idx
admissions = admissions.loc[idx]
admissions.head(5) #5811 rows

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,DAYS_DIFF
0,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,UNKNOWN,UNOBTAINABLE,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,0,1.14
12,34,115799,2186-07-18 16:46:00,2186-07-20 16:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,CHEST PAIN\CATH,0,1.97
34,363,196503,2176-03-01 15:26:00,2176-03-03 14:04:00,NEWBORN,CLINIC REFERRAL/PREMATURE,HOME,Government,UNKNOWN,PROTESTANT QUAKER,UNKNOWN,BLACK/AFRICAN AMERICAN,NEWBORN,0,1.94
56,55,190665,2136-04-03 18:35:00,2136-04-05 14:30:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,CAROTID STENOSIS\CAROTID ANGIOGRAM,0,1.83
61,60,198892,2146-11-12 14:58:00,2146-11-14 14:30:00,NEWBORN,CLINIC REFERRAL/PREMATURE,SHORT TERM HOSPITAL,Private,UNKNOWN,UNOBTAINABLE,UNKNOWN,UNKNOWN/NOT SPECIFIED,NEWBORN,0,1.98


In [10]:
#pre-processing for D_ICD_DIAGNOSES begins by deleting the non-essential columns
del d_icd_proc['SHORT_TITLE']
del d_icd_proc['ROW_ID']

In [11]:
d_icd_proc.head(5)

,ICD9_CODE,LONG_TITLE
0,851,Canthotomy
1,852,Blepharorrhaphy
2,859,Other adjustment of lid position
3,861,Reconstruction of eyelid with skin flap or graft
4,862,Reconstruction of eyelid with mucous membrane ...


In [12]:
#checking the proc_icd data frame
proc_icd

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,944,62641,154460,3,3404
1,945,2592,130856,1,9671
2,946,2592,130856,2,3893
3,947,55357,119355,1,9672
4,948,55357,119355,2,331
...,...,...,...,...,...
240090,228330,67415,150871,5,3736
240091,228331,67415,150871,6,3893
240092,228332,67415,150871,7,8872
240093,228333,67415,150871,8,3893


In [13]:
del proc_icd['ROW_ID']

In [14]:
#combining both D_ICD_PROC and PROC_ICD
proc = proc_icd.merge(d_icd_proc,on='ICD9_CODE',how='inner')

In [15]:
proc.head(5)

,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,LONG_TITLE
0,62641,154460,3,3404,Insertion of intercostal catheter for drainage
1,11143,101985,5,3404,Insertion of intercostal catheter for drainage
2,9736,160259,6,3404,Insertion of intercostal catheter for drainage
3,7546,147476,4,3404,Insertion of intercostal catheter for drainage
4,65535,178280,4,3404,Insertion of intercostal catheter for drainage


In [16]:
#locating the rows with seq_num = 1, because that gives the patients admission diagnosis
idy = np.where(proc['SEQ_NUM']==1)

In [17]:
#extracting the particular value to match up with idy
proc_title = proc.loc[idy]
proc_title_full = proc_title
proc_title

,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,LONG_TITLE
7,9370,160150,1,3404,Insertion of intercostal catheter for drainage
16,18852,142775,1,3404,Insertion of intercostal catheter for drainage
22,22533,145806,1,3404,Insertion of intercostal catheter for drainage
27,18874,101982,1,3404,Insertion of intercostal catheter for drainage
35,6038,111397,1,3404,Insertion of intercostal catheter for drainage
...,...,...,...,...,...
246142,43243,181417,1,3227,"Bronchoscopic bronchial thermoplasty, ablation..."
246144,62528,130949,1,1479,Other operations on vitreous
246150,51136,137832,1,8405,Amputation through forearm
246151,76479,106878,1,4583,Other and unspecified total intra-abdominal co...


In [18]:
#dropping the duplicates by keeping the first value since the ICD9_CODE has duplicates 
proc_title = proc_title.drop_duplicates(subset=['HADM_ID'],keep="first")

In [19]:
#dropping the non-essential columns
del proc_title['ICD9_CODE']
del proc_title['SEQ_NUM']

In [20]:
#merging with admissions table and proc_title to get the admission_diagnosis
admissions = admissions.merge(proc_title,on = ['HADM_ID','SUBJECT_ID'],how='left')
admissions #4429 rows

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,DAYS_DIFF,LONG_TITLE
0,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,UNKNOWN,UNOBTAINABLE,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,0,1.14,Insertion of endotracheal tube
1,34,115799,2186-07-18 16:46:00,2186-07-20 16:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,CHEST PAIN\CATH,0,1.97,Combined right and left heart cardiac catheter...
2,363,196503,2176-03-01 15:26:00,2176-03-03 14:04:00,NEWBORN,CLINIC REFERRAL/PREMATURE,HOME,Government,UNKNOWN,PROTESTANT QUAKER,UNKNOWN,BLACK/AFRICAN AMERICAN,NEWBORN,0,1.94,Prophylactic administration of vaccine against...
3,55,190665,2136-04-03 18:35:00,2136-04-05 14:30:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,CAROTID STENOSIS\CAROTID ANGIOGRAM,0,1.83,Percutaneous angioplasty of extracranial vesse...
4,60,198892,2146-11-12 14:58:00,2146-11-14 14:30:00,NEWBORN,CLINIC REFERRAL/PREMATURE,SHORT TERM HOSPITAL,Private,UNKNOWN,UNOBTAINABLE,UNKNOWN,UNKNOWN/NOT SPECIFIED,NEWBORN,0,1.98,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5705,98761,182540,2186-02-08 12:30:00,2186-02-08 20:21:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,SHORT GUT SYNDROME/SDA,0,0.33,Replacement of gastrostomy tube
5706,98768,127022,2112-09-15 09:45:00,2112-09-17 08:20:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,JEWISH,WIDOWED,WHITE,LEUKOCYTOSIS,1,1.94,NaN
5707,98797,105447,2132-12-24 20:06:00,2132-12-25 12:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,ALTERED MENTAL STATUS,1,0.66,Insertion of endotracheal tube
5708,98802,101071,2151-03-05 20:00:00,2151-03-06 09:10:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,SAH,1,0.55,Continuous invasive mechanical ventilation for...


In [21]:
#calling the compute_daily_averages function to take the count
var = compute_daily_averages(proc,'ICD9_CODE',admissions)
admissions= var

In [22]:
admissions.head(5) #4429 rows

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,DAYS_DIFF,LONG_TITLE,count
0,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,UNKNOWN,UNOBTAINABLE,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,0,1.14,Insertion of endotracheal tube,3.0
1,34,115799,2186-07-18 16:46:00,2186-07-20 16:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,CHEST PAIN\CATH,0,1.97,Combined right and left heart cardiac catheter...,3.0
2,363,196503,2176-03-01 15:26:00,2176-03-03 14:04:00,NEWBORN,CLINIC REFERRAL/PREMATURE,HOME,Government,UNKNOWN,PROTESTANT QUAKER,UNKNOWN,BLACK/AFRICAN AMERICAN,NEWBORN,0,1.94,Prophylactic administration of vaccine against...,1.0
3,55,190665,2136-04-03 18:35:00,2136-04-05 14:30:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,CAROTID STENOSIS\CAROTID ANGIOGRAM,0,1.83,Percutaneous angioplasty of extracranial vesse...,5.0
4,60,198892,2146-11-12 14:58:00,2146-11-14 14:30:00,NEWBORN,CLINIC REFERRAL/PREMATURE,SHORT TERM HOSPITAL,Private,UNKNOWN,UNOBTAINABLE,UNKNOWN,UNKNOWN/NOT SPECIFIED,NEWBORN,0,1.98,NaN,NaN


In [23]:
#renaming the count to NUM_PROCEDURES
admissions = admissions.rename(columns={'count': 'NUM_PROCEDURES'})

In [24]:
#rounding off the NUM_PROCEDURES
admissions['NUM_PROCEDURES'] = round(admissions['NUM_PROCEDURES']/admissions['DAYS_DIFF'],2)

In [25]:
admissions.head(5)

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,DAYS_DIFF,LONG_TITLE,NUM_PROCEDURES
0,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,UNKNOWN,UNOBTAINABLE,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,0,1.14,Insertion of endotracheal tube,2.63
1,34,115799,2186-07-18 16:46:00,2186-07-20 16:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,CHEST PAIN\CATH,0,1.97,Combined right and left heart cardiac catheter...,1.52
2,363,196503,2176-03-01 15:26:00,2176-03-03 14:04:00,NEWBORN,CLINIC REFERRAL/PREMATURE,HOME,Government,UNKNOWN,PROTESTANT QUAKER,UNKNOWN,BLACK/AFRICAN AMERICAN,NEWBORN,0,1.94,Prophylactic administration of vaccine against...,0.52
3,55,190665,2136-04-03 18:35:00,2136-04-05 14:30:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,CAROTID STENOSIS\CAROTID ANGIOGRAM,0,1.83,Percutaneous angioplasty of extracranial vesse...,2.73
4,60,198892,2146-11-12 14:58:00,2146-11-14 14:30:00,NEWBORN,CLINIC REFERRAL/PREMATURE,SHORT TERM HOSPITAL,Private,UNKNOWN,UNOBTAINABLE,UNKNOWN,UNKNOWN/NOT SPECIFIED,NEWBORN,0,1.98,NaN,NaN


In [26]:
#calling the compute_daily_averages function to take the count
var = compute_daily_averages(cpt,'CPT_CD',admissions)
admissions= var


admissions = admissions.rename(columns={'count': 'NUM_CPTEVENTS'})
admissions['NUM_CPTEVENTS'] = round(admissions['NUM_CPTEVENTS']/admissions['DAYS_DIFF'],2)

In [27]:
admissions.head(5)

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,DAYS_DIFF,LONG_TITLE,NUM_PROCEDURES,NUM_CPTEVENTS
0,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,UNKNOWN,UNOBTAINABLE,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,0,1.14,Insertion of endotracheal tube,2.63,NaN
1,34,115799,2186-07-18 16:46:00,2186-07-20 16:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,CHEST PAIN\CATH,0,1.97,Combined right and left heart cardiac catheter...,1.52,1.52
2,363,196503,2176-03-01 15:26:00,2176-03-03 14:04:00,NEWBORN,CLINIC REFERRAL/PREMATURE,HOME,Government,UNKNOWN,PROTESTANT QUAKER,UNKNOWN,BLACK/AFRICAN AMERICAN,NEWBORN,0,1.94,Prophylactic administration of vaccine against...,0.52,NaN
3,55,190665,2136-04-03 18:35:00,2136-04-05 14:30:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,CAROTID STENOSIS\CAROTID ANGIOGRAM,0,1.83,Percutaneous angioplasty of extracranial vesse...,2.73,NaN
4,60,198892,2146-11-12 14:58:00,2146-11-14 14:30:00,NEWBORN,CLINIC REFERRAL/PREMATURE,SHORT TERM HOSPITAL,Private,UNKNOWN,UNOBTAINABLE,UNKNOWN,UNKNOWN/NOT SPECIFIED,NEWBORN,0,1.98,NaN,NaN,NaN


In [28]:
var = compute_daily_averages(transfers,'ROW_ID',admissions)
admissions= var

In [29]:
admissions = admissions.rename(columns={'count': 'NUM_TRANSFERS'})
admissions['NUM_TRANSFERS'] = round(admissions['NUM_TRANSFERS']/admissions['DAYS_DIFF'],2)

In [30]:
admissions

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,DAYS_DIFF,LONG_TITLE,NUM_PROCEDURES,NUM_CPTEVENTS,NUM_TRANSFERS
0,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,UNKNOWN,UNOBTAINABLE,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,0,1.14,Insertion of endotracheal tube,2.63,NaN,1.75
1,34,115799,2186-07-18 16:46:00,2186-07-20 16:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,CHEST PAIN\CATH,0,1.97,Combined right and left heart cardiac catheter...,1.52,1.52,2.03
2,363,196503,2176-03-01 15:26:00,2176-03-03 14:04:00,NEWBORN,CLINIC REFERRAL/PREMATURE,HOME,Government,UNKNOWN,PROTESTANT QUAKER,UNKNOWN,BLACK/AFRICAN AMERICAN,NEWBORN,0,1.94,Prophylactic administration of vaccine against...,0.52,NaN,2.06
3,55,190665,2136-04-03 18:35:00,2136-04-05 14:30:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,CAROTID STENOSIS\CAROTID ANGIOGRAM,0,1.83,Percutaneous angioplasty of extracranial vesse...,2.73,NaN,1.09
4,60,198892,2146-11-12 14:58:00,2146-11-14 14:30:00,NEWBORN,CLINIC REFERRAL/PREMATURE,SHORT TERM HOSPITAL,Private,UNKNOWN,UNOBTAINABLE,UNKNOWN,UNKNOWN/NOT SPECIFIED,NEWBORN,0,1.98,NaN,NaN,NaN,1.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5705,98761,182540,2186-02-08 12:30:00,2186-02-08 20:21:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,SHORT GUT SYNDROME/SDA,0,0.33,Replacement of gastrostomy tube,3.03,NaN,9.09
5706,98768,127022,2112-09-15 09:45:00,2112-09-17 08:20:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,JEWISH,WIDOWED,WHITE,LEUKOCYTOSIS,1,1.94,NaN,NaN,1.55,1.03
5707,98797,105447,2132-12-24 20:06:00,2132-12-25 12:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,ALTERED MENTAL STATUS,1,0.66,Insertion of endotracheal tube,3.03,4.55,3.03
5708,98802,101071,2151-03-05 20:00:00,2151-03-06 09:10:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,SAH,1,0.55,Continuous invasive mechanical ventilation for...,1.82,NaN,3.64


In [31]:
labVar = compute_daily_averages(lab,'ITEMID',admissions)
admissions= labVar

In [32]:
admissions = admissions.rename(columns={'count': 'NUM_LABITEMS'})
admissions['NUM_LABITEMS'] = round(admissions['NUM_LABITEMS']/admissions['DAYS_DIFF'],2)

In [33]:
admissions.tail(5)

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,DAYS_DIFF,LONG_TITLE,NUM_PROCEDURES,NUM_CPTEVENTS,NUM_TRANSFERS,NUM_LABITEMS
5705,98761,182540,2186-02-08 12:30:00,2186-02-08 20:21:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,SHORT GUT SYNDROME/SDA,0,0.33,Replacement of gastrostomy tube,3.03,NaN,9.09,NaN
5706,98768,127022,2112-09-15 09:45:00,2112-09-17 08:20:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,JEWISH,WIDOWED,WHITE,LEUKOCYTOSIS,1,1.94,NaN,NaN,1.55,1.03,47.42
5707,98797,105447,2132-12-24 20:06:00,2132-12-25 12:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,ALTERED MENTAL STATUS,1,0.66,Insertion of endotracheal tube,3.03,4.55,3.03,178.79
5708,98802,101071,2151-03-05 20:00:00,2151-03-06 09:10:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,SAH,1,0.55,Continuous invasive mechanical ventilation for...,1.82,NaN,3.64,100.00
5709,98813,190264,2131-10-25 03:09:00,2131-10-26 17:44:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,CATHOLIC,MARRIED,WHITE,INTRACRANIAL HEMORRHAGE,0,1.61,NaN,NaN,1.24,1.24,45.96


In [34]:
MicrolabVar = compute_daily_averages(microLab,'SPEC_ITEMID',admissions)
admissions= MicrolabVar

In [35]:
admissions = admissions.rename(columns={'count': 'NUM_MICROLABS'})
admissions['NUM_MICROLABS'] = round(admissions['NUM_MICROLABS']/admissions['DAYS_DIFF'],2)

In [36]:
admissions.head(5)

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,...,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,DAYS_DIFF,LONG_TITLE,NUM_PROCEDURES,NUM_CPTEVENTS,NUM_TRANSFERS,NUM_LABITEMS,NUM_MICROLABS
0,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,UNKNOWN,UNOBTAINABLE,...,WHITE,BENZODIAZEPINE OVERDOSE,0,1.14,Insertion of endotracheal tube,2.63,NaN,1.75,79.82,0.88
1,34,115799,2186-07-18 16:46:00,2186-07-20 16:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,ENGL,CATHOLIC,...,WHITE,CHEST PAIN\CATH,0,1.97,Combined right and left heart cardiac catheter...,1.52,1.52,2.03,50.76,NaN
2,363,196503,2176-03-01 15:26:00,2176-03-03 14:04:00,NEWBORN,CLINIC REFERRAL/PREMATURE,HOME,Government,UNKNOWN,PROTESTANT QUAKER,...,BLACK/AFRICAN AMERICAN,NEWBORN,0,1.94,Prophylactic administration of vaccine against...,0.52,NaN,2.06,NaN,NaN
3,55,190665,2136-04-03 18:35:00,2136-04-05 14:30:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicare,ENGL,CATHOLIC,...,WHITE,CAROTID STENOSIS\CAROTID ANGIOGRAM,0,1.83,Percutaneous angioplasty of extracranial vesse...,2.73,NaN,1.09,26.78,NaN
4,60,198892,2146-11-12 14:58:00,2146-11-14 14:30:00,NEWBORN,CLINIC REFERRAL/PREMATURE,SHORT TERM HOSPITAL,Private,UNKNOWN,UNOBTAINABLE,...,UNKNOWN/NOT SPECIFIED,NEWBORN,0,1.98,NaN,NaN,NaN,1.52,1.52,NaN


In [37]:
NoteVar = compute_daily_averages(noteEvents,'HADM_ID',admissions)
admissions= NoteVar

In [38]:
admissions = admissions.rename(columns={'count': 'NUM_NOTEEVENTS'})
admissions['NUM_NOTEEVENTS'] = round(admissions['NUM_NOTEEVENTS']/admissions['DAYS_DIFF'],2)

In [39]:
admissions

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,...,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,DAYS_DIFF,LONG_TITLE,NUM_PROCEDURES,NUM_CPTEVENTS,NUM_TRANSFERS,NUM_LABITEMS,NUM_MICROLABS,NUM_NOTEEVENTS
0,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,UNKNOWN,UNOBTAINABLE,...,BENZODIAZEPINE OVERDOSE,0,1.14,Insertion of endotracheal tube,2.63,NaN,1.75,79.82,0.88,8.77
1,34,115799,2186-07-18 16:46:00,2186-07-20 16:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,ENGL,CATHOLIC,...,CHEST PAIN\CATH,0,1.97,Combined right and left heart cardiac catheter...,1.52,1.52,2.03,50.76,NaN,4.06
2,363,196503,2176-03-01 15:26:00,2176-03-03 14:04:00,NEWBORN,CLINIC REFERRAL/PREMATURE,HOME,Government,UNKNOWN,PROTESTANT QUAKER,...,NEWBORN,0,1.94,Prophylactic administration of vaccine against...,0.52,NaN,2.06,NaN,NaN,1.55
3,55,190665,2136-04-03 18:35:00,2136-04-05 14:30:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicare,ENGL,CATHOLIC,...,CAROTID STENOSIS\CAROTID ANGIOGRAM,0,1.83,Percutaneous angioplasty of extracranial vesse...,2.73,NaN,1.09,26.78,NaN,3.83
4,60,198892,2146-11-12 14:58:00,2146-11-14 14:30:00,NEWBORN,CLINIC REFERRAL/PREMATURE,SHORT TERM HOSPITAL,Private,UNKNOWN,UNOBTAINABLE,...,NEWBORN,0,1.98,NaN,NaN,NaN,1.52,1.52,NaN,6.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5705,98761,182540,2186-02-08 12:30:00,2186-02-08 20:21:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Medicare,ENGL,CATHOLIC,...,SHORT GUT SYNDROME/SDA,0,0.33,Replacement of gastrostomy tube,3.03,NaN,9.09,NaN,NaN,6.06
5706,98768,127022,2112-09-15 09:45:00,2112-09-17 08:20:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,JEWISH,...,LEUKOCYTOSIS,1,1.94,NaN,NaN,1.55,1.03,47.42,2.06,1.55
5707,98797,105447,2132-12-24 20:06:00,2132-12-25 12:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,...,ALTERED MENTAL STATUS,1,0.66,Insertion of endotracheal tube,3.03,4.55,3.03,178.79,9.09,NaN
5708,98802,101071,2151-03-05 20:00:00,2151-03-06 09:10:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,...,SAH,1,0.55,Continuous invasive mechanical ventilation for...,1.82,NaN,3.64,100.00,1.82,1.82


In [40]:
outputVar = compute_daily_averages(outputEvents,'ITEMID',admissions)
admissions= outputVar

In [41]:
admissions = admissions.rename(columns={'count': 'NUM_OUTPUT'})
admissions['NUM_OUTPUT'] = round(admissions['NUM_OUTPUT']/admissions['DAYS_DIFF'],2)

In [42]:
admissions

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,...,HOSPITAL_EXPIRE_FLAG,DAYS_DIFF,LONG_TITLE,NUM_PROCEDURES,NUM_CPTEVENTS,NUM_TRANSFERS,NUM_LABITEMS,NUM_MICROLABS,NUM_NOTEEVENTS,NUM_OUTPUT
0,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,UNKNOWN,UNOBTAINABLE,...,0,1.14,Insertion of endotracheal tube,2.63,NaN,1.75,79.82,0.88,8.77,6.14
1,34,115799,2186-07-18 16:46:00,2186-07-20 16:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,ENGL,CATHOLIC,...,0,1.97,Combined right and left heart cardiac catheter...,1.52,1.52,2.03,50.76,NaN,4.06,4.57
2,363,196503,2176-03-01 15:26:00,2176-03-03 14:04:00,NEWBORN,CLINIC REFERRAL/PREMATURE,HOME,Government,UNKNOWN,PROTESTANT QUAKER,...,0,1.94,Prophylactic administration of vaccine against...,0.52,NaN,2.06,NaN,NaN,1.55,NaN
3,55,190665,2136-04-03 18:35:00,2136-04-05 14:30:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicare,ENGL,CATHOLIC,...,0,1.83,Percutaneous angioplasty of extracranial vesse...,2.73,NaN,1.09,26.78,NaN,3.83,7.65
4,60,198892,2146-11-12 14:58:00,2146-11-14 14:30:00,NEWBORN,CLINIC REFERRAL/PREMATURE,SHORT TERM HOSPITAL,Private,UNKNOWN,UNOBTAINABLE,...,0,1.98,NaN,NaN,NaN,1.52,1.52,NaN,6.06,0.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5705,98761,182540,2186-02-08 12:30:00,2186-02-08 20:21:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Medicare,ENGL,CATHOLIC,...,0,0.33,Replacement of gastrostomy tube,3.03,NaN,9.09,NaN,NaN,6.06,NaN
5706,98768,127022,2112-09-15 09:45:00,2112-09-17 08:20:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,JEWISH,...,1,1.94,NaN,NaN,1.55,1.03,47.42,2.06,1.55,4.12
5707,98797,105447,2132-12-24 20:06:00,2132-12-25 12:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,...,1,0.66,Insertion of endotracheal tube,3.03,4.55,3.03,178.79,9.09,NaN,18.18
5708,98802,101071,2151-03-05 20:00:00,2151-03-06 09:10:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,...,1,0.55,Continuous invasive mechanical ventilation for...,1.82,NaN,3.64,100.00,1.82,1.82,9.09


In [43]:

outputVar = compute_daily_averages(diag_icd,'ICD9_CODE',admissions)
admissions= outputVar


In [44]:
admissions = admissions.rename(columns={'count': 'NUM_DIAGNOSIS'})
admissions['NUM_DIAGNOSIS'] = round(admissions['NUM_DIAGNOSIS']/admissions['DAYS_DIFF'],2)

In [45]:
admissions.head(5)

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,...,DAYS_DIFF,LONG_TITLE,NUM_PROCEDURES,NUM_CPTEVENTS,NUM_TRANSFERS,NUM_LABITEMS,NUM_MICROLABS,NUM_NOTEEVENTS,NUM_OUTPUT,NUM_DIAGNOSIS
0,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,UNKNOWN,UNOBTAINABLE,...,1.14,Insertion of endotracheal tube,2.63,NaN,1.75,79.82,0.88,8.77,6.14,6.14
1,34,115799,2186-07-18 16:46:00,2186-07-20 16:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,ENGL,CATHOLIC,...,1.97,Combined right and left heart cardiac catheter...,1.52,1.52,2.03,50.76,NaN,4.06,4.57,4.06
2,363,196503,2176-03-01 15:26:00,2176-03-03 14:04:00,NEWBORN,CLINIC REFERRAL/PREMATURE,HOME,Government,UNKNOWN,PROTESTANT QUAKER,...,1.94,Prophylactic administration of vaccine against...,0.52,NaN,2.06,NaN,NaN,1.55,NaN,1.55
3,55,190665,2136-04-03 18:35:00,2136-04-05 14:30:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicare,ENGL,CATHOLIC,...,1.83,Percutaneous angioplasty of extracranial vesse...,2.73,NaN,1.09,26.78,NaN,3.83,7.65,5.46
4,60,198892,2146-11-12 14:58:00,2146-11-14 14:30:00,NEWBORN,CLINIC REFERRAL/PREMATURE,SHORT TERM HOSPITAL,Private,UNKNOWN,UNOBTAINABLE,...,1.98,NaN,NaN,NaN,1.52,1.52,NaN,6.06,0.51,1.52


In [46]:
patients = patients[['SUBJECT_ID','GENDER','DOB']]

In [47]:
admissions = admissions.merge(patients,on = ['SUBJECT_ID'],how='left')

In [48]:
admissions

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,...,NUM_PROCEDURES,NUM_CPTEVENTS,NUM_TRANSFERS,NUM_LABITEMS,NUM_MICROLABS,NUM_NOTEEVENTS,NUM_OUTPUT,NUM_DIAGNOSIS,GENDER,DOB
0,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,UNKNOWN,UNOBTAINABLE,...,2.63,NaN,1.75,79.82,0.88,8.77,6.14,6.14,F,2131-05-07 00:00:00
1,34,115799,2186-07-18 16:46:00,2186-07-20 16:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,ENGL,CATHOLIC,...,1.52,1.52,2.03,50.76,NaN,4.06,4.57,4.06,M,1886-07-18 00:00:00
2,363,196503,2176-03-01 15:26:00,2176-03-03 14:04:00,NEWBORN,CLINIC REFERRAL/PREMATURE,HOME,Government,UNKNOWN,PROTESTANT QUAKER,...,0.52,NaN,2.06,NaN,NaN,1.55,NaN,1.55,F,2176-03-01 00:00:00
3,55,190665,2136-04-03 18:35:00,2136-04-05 14:30:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicare,ENGL,CATHOLIC,...,2.73,NaN,1.09,26.78,NaN,3.83,7.65,5.46,F,2072-02-04 00:00:00
4,60,198892,2146-11-12 14:58:00,2146-11-14 14:30:00,NEWBORN,CLINIC REFERRAL/PREMATURE,SHORT TERM HOSPITAL,Private,UNKNOWN,UNOBTAINABLE,...,NaN,NaN,1.52,1.52,NaN,6.06,0.51,1.52,F,2146-11-12 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5705,98761,182540,2186-02-08 12:30:00,2186-02-08 20:21:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Medicare,ENGL,CATHOLIC,...,3.03,NaN,9.09,NaN,NaN,6.06,NaN,42.42,F,2138-09-16 00:00:00
5706,98768,127022,2112-09-15 09:45:00,2112-09-17 08:20:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,JEWISH,...,NaN,1.55,1.03,47.42,2.06,1.55,4.12,7.22,F,2027-07-26 00:00:00
5707,98797,105447,2132-12-24 20:06:00,2132-12-25 12:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,...,3.03,4.55,3.03,178.79,9.09,NaN,18.18,13.64,M,2044-12-27 00:00:00
5708,98802,101071,2151-03-05 20:00:00,2151-03-06 09:10:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,...,1.82,NaN,3.64,100.00,1.82,1.82,9.09,7.27,F,2067-09-21 00:00:00


In [49]:
admissions['ADMITTIME'] = pd.to_datetime(admissions['ADMITTIME']).dt.date
admissions['DOB'] = pd.to_datetime(admissions['DOB']).dt.date
admissions['age'] = admissions.apply(lambda e: (e['ADMITTIME'] - e['DOB']).days/365, axis=1)

In [50]:
admissions

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,...,NUM_CPTEVENTS,NUM_TRANSFERS,NUM_LABITEMS,NUM_MICROLABS,NUM_NOTEEVENTS,NUM_OUTPUT,NUM_DIAGNOSIS,GENDER,DOB,age
0,22,165315,2196-04-09,2196-04-10 15:54:00,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,UNKNOWN,UNOBTAINABLE,...,NaN,1.75,79.82,0.88,8.77,6.14,6.14,F,2131-05-07,64.969863
1,34,115799,2186-07-18,2186-07-20 16:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,ENGL,CATHOLIC,...,1.52,2.03,50.76,NaN,4.06,4.57,4.06,M,1886-07-18,300.200000
2,363,196503,2176-03-01,2176-03-03 14:04:00,NEWBORN,CLINIC REFERRAL/PREMATURE,HOME,Government,UNKNOWN,PROTESTANT QUAKER,...,NaN,2.06,NaN,NaN,1.55,NaN,1.55,F,2176-03-01,0.000000
3,55,190665,2136-04-03,2136-04-05 14:30:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicare,ENGL,CATHOLIC,...,NaN,1.09,26.78,NaN,3.83,7.65,5.46,F,2072-02-04,64.202740
4,60,198892,2146-11-12,2146-11-14 14:30:00,NEWBORN,CLINIC REFERRAL/PREMATURE,SHORT TERM HOSPITAL,Private,UNKNOWN,UNOBTAINABLE,...,NaN,1.52,1.52,NaN,6.06,0.51,1.52,F,2146-11-12,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5705,98761,182540,2186-02-08,2186-02-08 20:21:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Medicare,ENGL,CATHOLIC,...,NaN,9.09,NaN,NaN,6.06,NaN,42.42,F,2138-09-16,47.430137
5706,98768,127022,2112-09-15,2112-09-17 08:20:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,JEWISH,...,1.55,1.03,47.42,2.06,1.55,4.12,7.22,F,2027-07-26,85.197260
5707,98797,105447,2132-12-24,2132-12-25 12:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,...,4.55,3.03,178.79,9.09,NaN,18.18,13.64,M,2044-12-27,88.049315
5708,98802,101071,2151-03-05,2151-03-06 09:10:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,...,NaN,3.64,100.00,1.82,1.82,9.09,7.27,F,2067-09-21,83.506849


In [51]:
admissions['age'] = admissions['age'].astype(int)

In [52]:
del admissions['ADMITTIME']
del admissions['DISCHTIME']


In [53]:
first_column = admissions.pop('GENDER')

In [54]:
admissions.insert(2, 'GENDER', first_column)

In [55]:
age= admissions.pop('age')
admissions.insert(3, 'age', age)

In [56]:
del admissions['SUBJECT_ID']

In [57]:
del admissions['DOB']

In [58]:
admissions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5710 entries, 0 to 5709
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   HADM_ID               5710 non-null   int64  
 1   GENDER                5710 non-null   object 
 2   age                   5710 non-null   int32  
 3   ADMISSION_TYPE        5710 non-null   object 
 4   ADMISSION_LOCATION    5710 non-null   object 
 5   DISCHARGE_LOCATION    5710 non-null   object 
 6   INSURANCE             5710 non-null   object 
 7   LANGUAGE              5710 non-null   object 
 8   RELIGION              5710 non-null   object 
 9   MARITAL_STATUS        5710 non-null   object 
 10  ETHNICITY             5710 non-null   object 
 11  DIAGNOSIS             5688 non-null   object 
 12  HOSPITAL_EXPIRE_FLAG  5710 non-null   int64  
 13  DAYS_DIFF             5710 non-null   float64
 14  LONG_TITLE            4172 non-null   object 
 15  NUM_PROCEDURES       

In [59]:
#imputing Nan values in numerical columns
column_list=['NUM_PROCEDURES','NUM_CPTEVENTS','NUM_TRANSFERS','NUM_LABITEMS','NUM_MICROLABS','NUM_NOTEEVENTS','NUM_OUTPUT','NUM_DIAGNOSIS']
for col in column_list:
    admissions[col] = admissions[col].fillna(0)

In [60]:
admissions["TotalEvents"] =admissions[column_list].sum(axis=1)

In [61]:
admissions

,HADM_ID,GENDER,age,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,...,LONG_TITLE,NUM_PROCEDURES,NUM_CPTEVENTS,NUM_TRANSFERS,NUM_LABITEMS,NUM_MICROLABS,NUM_NOTEEVENTS,NUM_OUTPUT,NUM_DIAGNOSIS,TotalEvents
0,165315,F,64,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,UNKNOWN,UNOBTAINABLE,MARRIED,...,Insertion of endotracheal tube,2.63,0.00,1.75,79.82,0.88,8.77,6.14,6.14,106.13
1,115799,M,300,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,ENGL,CATHOLIC,MARRIED,...,Combined right and left heart cardiac catheter...,1.52,1.52,2.03,50.76,0.00,4.06,4.57,4.06,68.52
2,196503,F,0,NEWBORN,CLINIC REFERRAL/PREMATURE,HOME,Government,UNKNOWN,PROTESTANT QUAKER,UNKNOWN,...,Prophylactic administration of vaccine against...,0.52,0.00,2.06,0.00,0.00,1.55,0.00,1.55,5.68
3,190665,F,64,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicare,ENGL,CATHOLIC,MARRIED,...,Percutaneous angioplasty of extracranial vesse...,2.73,0.00,1.09,26.78,0.00,3.83,7.65,5.46,47.54
4,198892,F,0,NEWBORN,CLINIC REFERRAL/PREMATURE,SHORT TERM HOSPITAL,Private,UNKNOWN,UNOBTAINABLE,UNKNOWN,...,NaN,0.00,0.00,1.52,1.52,0.00,6.06,0.51,1.52,11.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5705,182540,F,47,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Medicare,ENGL,CATHOLIC,MARRIED,...,Replacement of gastrostomy tube,3.03,0.00,9.09,0.00,0.00,6.06,0.00,42.42,60.60
5706,127022,F,85,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,JEWISH,WIDOWED,...,NaN,0.00,1.55,1.03,47.42,2.06,1.55,4.12,7.22,64.95
5707,105447,M,88,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,...,Insertion of endotracheal tube,3.03,4.55,3.03,178.79,9.09,0.00,18.18,13.64,230.31
5708,101071,F,83,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,...,Continuous invasive mechanical ventilation for...,1.82,0.00,3.64,100.00,1.82,1.82,9.09,7.27,125.46


In [62]:
admissions = admissions.rename(columns={'LONG_TITLE':'ADMIT_PROCEDURE','age':'AGE','TotalEvents':'TOTAL_EVENTS'})

In [63]:
admissions['ADMIT_PROCEDURE'] = admissions['ADMIT_PROCEDURE'].fillna('NOT AVAILABLE')
admissions['DIAGNOSIS'] = admissions['DIAGNOSIS'].fillna('NOT AVAILABLE')

In [64]:
admissions['AGE'].unique()

array([ 64, 300,   0,  88,  73,  85,  44,  60,  62,  22,  23,  24,  25,
        69, 302,  26,  66,  55,  42,  21,  87,  40,  51,  78,  82,  50,
        79,  70,  33,  57,  83,  75,  56,  32,  46,  81,  68,  49,  59,
        19,  52,  65,  45,  37,  61,  63,  74,  39,  84,  20,  72,  38,
        67,  77,  48,  43,  54,  86,  53,  41,  34,  30,  47,  71,  29,
        80,  58,  18,  76,  36,  28,  35,  89, 304,  27, 301,  31, 310,
        17,  16,  15, 308, 306, 305, 307, 303,  14])

In [65]:
admissions.loc[admissions.AGE.between(300,350),'AGE']=np.nan


In [66]:
admissions['AGE'] = admissions['AGE'].fillna((admissions['AGE'].median()))



In [67]:
admissions['AGE'] = admissions['AGE'].astype(int)


In [68]:
import string
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [69]:
admissions['DIAGNOSIS'] = admissions['DIAGNOSIS'].apply(remove_punctuations)

In [70]:
admissions

,HADM_ID,GENDER,AGE,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,...,ADMIT_PROCEDURE,NUM_PROCEDURES,NUM_CPTEVENTS,NUM_TRANSFERS,NUM_LABITEMS,NUM_MICROLABS,NUM_NOTEEVENTS,NUM_OUTPUT,NUM_DIAGNOSIS,TOTAL_EVENTS
0,165315,F,64,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,UNKNOWN,UNOBTAINABLE,MARRIED,...,Insertion of endotracheal tube,2.63,0.00,1.75,79.82,0.88,8.77,6.14,6.14,106.13
1,115799,M,53,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,ENGL,CATHOLIC,MARRIED,...,Combined right and left heart cardiac catheter...,1.52,1.52,2.03,50.76,0.00,4.06,4.57,4.06,68.52
2,196503,F,0,NEWBORN,CLINIC REFERRAL/PREMATURE,HOME,Government,UNKNOWN,PROTESTANT QUAKER,UNKNOWN,...,Prophylactic administration of vaccine against...,0.52,0.00,2.06,0.00,0.00,1.55,0.00,1.55,5.68
3,190665,F,64,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicare,ENGL,CATHOLIC,MARRIED,...,Percutaneous angioplasty of extracranial vesse...,2.73,0.00,1.09,26.78,0.00,3.83,7.65,5.46,47.54
4,198892,F,0,NEWBORN,CLINIC REFERRAL/PREMATURE,SHORT TERM HOSPITAL,Private,UNKNOWN,UNOBTAINABLE,UNKNOWN,...,NOT AVAILABLE,0.00,0.00,1.52,1.52,0.00,6.06,0.51,1.52,11.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5705,182540,F,47,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Medicare,ENGL,CATHOLIC,MARRIED,...,Replacement of gastrostomy tube,3.03,0.00,9.09,0.00,0.00,6.06,0.00,42.42,60.60
5706,127022,F,85,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,JEWISH,WIDOWED,...,NOT AVAILABLE,0.00,1.55,1.03,47.42,2.06,1.55,4.12,7.22,64.95
5707,105447,M,88,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,...,Insertion of endotracheal tube,3.03,4.55,3.03,178.79,9.09,0.00,18.18,13.64,230.31
5708,101071,F,83,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,...,Continuous invasive mechanical ventilation for...,1.82,0.00,3.64,100.00,1.82,1.82,9.09,7.27,125.46


In [72]:
admissions.to_csv('mimic_new.csv')